In [1]:
import os
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
import speech_recognition
from gtts import gTTS
from googletrans import Translator, LANGUAGES
from textblob import TextBlob
from langdetect import detect
from langcodes import Language
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from googletrans import LANGUAGES
import googletrans

# Text to Text Conversion Functions
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    cleaned_tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

def detect_language_and_name(text):
    try:
        preprocessed_text = preprocess_text(text)
        language_code = detect(preprocessed_text)
        language_name = Language.get(language_code).language_name('en')
        return language_name
    except Exception as e:
        print(f"Error: {e}")
        return None

# Voice to Text Conversion Functions
def voice_to_text(target_language):
    recognizer = speech_recognition.Recognizer()
    with speech_recognition.Microphone() as source:
        print("Speak now")
        voice = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(voice, language='en-US')  # Recognize speech in English
            print("You said:", text)

            translator = googletrans.Translator()
            translation = translator.translate(text, dest=target_language)
            print("Translation:", translation.text)
            return translation.text
        except speech_recognition.UnknownValueError:
            print("Sorry, could not understand audio.")
            return None
        except speech_recognition.RequestError as e:
            print("Error:", e)
            return None

# Voice to Voice Conversion Functions
def voice_to_voice(target_language):
    recognizer = speech_recognition.Recognizer()
    with speech_recognition.Microphone() as source:
        print("Speak now")
        voice = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(voice, language='en')  # Recognize speech in English (for simplicity)
            print("You said:", text)

            translator = Translator()
            translation = translator.translate(text, dest=target_language)
            print("Translation:", translation.text)

            # Get the corresponding gTTS language code
            gtts_language_code = map_language_to_gtts_code(target_language)
            
            # Convert translated text to speech using gTTS
            tts_translation = gTTS(text=translation.text, lang=gtts_language_code)
            tts_translation.save("translated_output.mp3")

            # Play the translated text
            display(Audio("translated_output.mp3", autoplay=True))
            clear_output(wait=True)
        except speech_recognition.UnknownValueError:
            print("Sorry, could not understand audio.")
        except speech_recognition.RequestError as e:
            print("Error:", e)
        except Exception as e:
            print("Error:", e)

# Emotion in Speech Functions
def get_emotion(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    if sentiment_score > 0.3:
        return "Joy/Happiness"
    elif sentiment_score < -0.3:
        return "Sadness"
    elif sentiment_score < 0:
        return "Anger"
    elif sentiment_score > 0:
        return "Surprise"
    elif sentiment_score < 0.3:
        return "Fear"
    else:
        return "Neutral"

def emotion_in_speech():
    recognizer = speech_recognition.Recognizer()
    with speech_recognition.Microphone() as source:
        print("Speak now")
        voice = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(voice, language='en')  # Recognize speech in English (for simplicity)
            print("You said:", text)

            # Determine emotion
            emotion = get_emotion(text)
            print("Emotion:", emotion)

            # Convert emotion to speech using gTTS
            tts_emotion = gTTS(text=f"The speaker is expressing {emotion}.", lang='en')
            tts_emotion.save("emotion_output.mp3")

            # Play the emotion
            display(Audio("emotion_output.mp3", autoplay=True))
            clear_output(wait=True)
        except speech_recognition.UnknownValueError:
            print("Sorry, could not understand audio.")
        except speech_recognition.RequestError as e:
            print("Error:", e)
        except Exception as e:
            print("Error:", e)

# Function to map language name to gTTS language code
def map_language_to_gtts_code(language_name):
    gtts_codes = {
        'Afrikaans': 'af', 'Albanian': 'sq', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az',
        'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg',
        'Catalan': 'ca', 'Cebuano': 'ceb', 'Chichewa': 'ny', 'Chinese (Simplified)': 'zh-cn',
        'Chinese (Traditional)': 'zh-tw', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs',
        'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et',
        'Filipino': 'fil', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl',
        'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht',
        'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'he', 'Hindi': 'hi', 'Hmong': 'hmn',
        'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga',
        'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk',
        'Khmer': 'km', 'Korean': 'ko', 'Kurdish (Kurmanji)': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo',
        'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk',
        'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi',
        'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no',
        'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese': 'pt',
        'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd',
        'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si',
        'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su',
        'Swahili': 'sw', 'Swedish': 'sv', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th',
        'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy',
        'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'
    }
    return gtts_codes.get(language_name.lower(), 'en')

# Main Conversion Function
def convert(option, target_language):
    if option == "Text to Text":
        input_text_widget = widgets.Textarea(
            value='',
            placeholder='Enter text',
            description='Input Text:',
            disabled=False
        )
        display(input_text_widget)

        # Define a function to handle button click for text-to-text conversion
        def on_text_to_text_button_click(b):
            input_text = input_text_widget.value
            detected_language = detect_language_and_name(input_text)
            print("Detected Language:", detected_language)
            print("Selected Target Language:", target_language)
            translator = Translator()
            translation = translator.translate(input_text, dest=target_language)
            print("Translated Text:", translation.text)

        # Create a button for manual triggering of text-to-text conversion
        text_to_text_button = widgets.Button(description="Translate")
        text_to_text_button.on_click(on_text_to_text_button_click)
        display(text_to_text_button)

    elif option == "Voice to Text":
        speak_now_button = widgets.Button(description="Speak now")
        display(speak_now_button)

        def on_speak_now_button_click(b):
            translated_text = voice_to_text(target_language)
            if translated_text:
                print("Translated Text:", translated_text)

        speak_now_button.on_click(on_speak_now_button_click)

        # Display Emotion Button
        display(emotion_button)

    elif option == "Voice to Voice":
        speak_now_button = widgets.Button(description="Speak now")
        display(speak_now_button)

        def on_speak_now_button_click(b):
            voice_to_voice(target_language)

        speak_now_button.on_click(on_speak_now_button_click)

        # Display Emotion Button
        display(emotion_button)

# Function for Emotion Button Click
def on_emotion_button_click(b):
    clear_output(wait=True)
    display(conversion_dropdown)
    display(language_dropdown)
    emotion_in_speech()

# Dropdown menu for conversion type
conversion_options = ["Text to Text", "Voice to Text", "Voice to Voice"]
conversion_dropdown = widgets.Dropdown(
    options=conversion_options,
    description='Conversion Type:'
)

# Dropdown menu for target language
languages = list(LANGUAGES.values())
default_language = languages[0] if languages else 'English'  # Set default to the first language or 'English'
language_dropdown = widgets.Dropdown(
    options=languages,
    value=default_language,
    description='Target Language:'
)

# Emotion Button
emotion_button = widgets.Button(description="Check Emotion")
emotion_button.on_click(on_emotion_button_click)

# Function to handle dropdown change
def on_dropdown_change(change):
    clear_output(wait=True)
    display(conversion_dropdown)
    display(language_dropdown)
    convert(conversion_dropdown.value, language_dropdown.value)

# Assigning the function to the dropdown change event
conversion_dropdown.observe(on_dropdown_change, names='value')

# Display the conversion dropdown and language dropdown
display(conversion_dropdown)
display(language_dropdown)


Dropdown(description='Conversion Type:', options=('Text to Text', 'Voice to Text', 'Voice to Voice'), value='T…

Dropdown(description='Target Language:', options=('afrikaans', 'albanian', 'amharic', 'arabic', 'armenian', 'a…

In [3]:
import time

# Function to measure the execution time of a given function
def measure_execution_time(func, *args, **kwargs):
    start_time = time.time()
    func(*args, **kwargs)
    end_time = time.time()
    return end_time - start_time

# Text to Text Conversion Efficiency
def text_to_text_conversion_efficiency(translator, target_language):
    input_text = "Hello, how are you?"
    
    start_time = time.time()
    detected_language = detect_language_and_name(input_text)
    translation = translator.translate(input_text, dest=target_language)
    end_time = time.time()

    execution_time = end_time - start_time
    print("Text to Text Conversion Efficiency:", execution_time)

# Voice to Text Conversion Efficiency
def voice_to_text_conversion_efficiency(target_language):
    start_time = time.time()
    translated_text = voice_to_text(target_language)
    end_time = time.time()

    execution_time = end_time - start_time
    print("Voice to Text Conversion Efficiency:", execution_time)

# Voice to Voice Conversion Efficiency
def voice_to_voice_conversion_efficiency(target_language):
    start_time = time.time()
    voice_to_voice(target_language)
    end_time = time.time()

    execution_time = end_time - start_time
    print("Voice to Voice Conversion Efficiency:", execution_time)

# Emotion in Speech Efficiency
def emotion_in_speech_efficiency():
    start_time = time.time()
    emotion_in_speech()
    end_time = time.time()

    execution_time = end_time - start_time
    print("Emotion in Speech Efficiency:", execution_time)

# Measure efficiency for each task
translator = Translator()
target_language = 'fr'  # Example target language
text_to_text_conversion_efficiency(translator, target_language)
voice_to_text_conversion_efficiency(target_language)
voice_to_voice_conversion_efficiency(target_language)
emotion_in_speech_efficiency()


Text to Text Conversion Efficiency: 2.4462451934814453
Speak now
Sorry, could not understand audio.
Voice to Text Conversion Efficiency: 6.430734157562256
Speak now
Sorry, could not understand audio.
Voice to Voice Conversion Efficiency: 6.083340883255005
Speak now
Sorry, could not understand audio.
Emotion in Speech Efficiency: 11.625463962554932


In [5]:
import time

# Function to measure the execution time of a given function
def measure_execution_time(func, *args, **kwargs):
    start_time = time.time()
    func(*args, **kwargs)
    end_time = time.time()
    return end_time - start_time

# Text to Text Conversion Efficiency
def text_to_text_conversion_efficiency(translator, target_language):
    input_text = "Hello, how are you?"
    
    start_time = time.time()
    detected_language = detect_language_and_name(input_text)
    translation = translator.translate(input_text, dest=target_language)
    end_time = time.time()

    execution_time = end_time - start_time
    print("Text to Text Conversion Efficiency:", execution_time)

# Voice to Text Conversion Efficiency
def voice_to_text_conversion_efficiency(target_language):
    start_time = time.time()
    translated_text = voice_to_text(target_language)
    end_time = time.time()

    execution_time = end_time - start_time
    print("Voice to Text Conversion Efficiency:", execution_time)

# Voice to Voice Conversion Efficiency
def voice_to_voice_conversion_efficiency(target_language):
    start_time = time.time()
    voice_to_voice(target_language)
    end_time = time.time()

    execution_time = end_time - start_time
    print("Voice to Voice Conversion Efficiency:", execution_time)

# Emotion in Speech Efficiency
def emotion_in_speech_efficiency():
    start_time = time.time()
    emotion_in_speech()
    end_time = time.time()

    execution_time = end_time - start_time
    print("Emotion in Speech Efficiency:", execution_time)

# Measure efficiency for each task
translator = Translator()
target_language = 'fr'  # Example target language
text_to_text_conversion_efficiency(translator, target_language)
voice_to_text_conversion_efficiency(target_language)
voice_to_voice_conversion_efficiency(target_language)
emotion_in_speech_efficiency()


Voice to Voice Conversion Efficiency: 15.320476293563843
Speak now
Sorry, could not understand audio.
Emotion in Speech Efficiency: 15.349382162094116
